In [1]:
from __future__ import print_function
import numpy as np
import pickle

In [2]:
# Tic Tac Toe problem
BOARD_ROWS = 3
BOARD_COLS = 3
BOARD_SIZE = BOARD_ROWS * BOARD_COLS

In [3]:
class State:
    def __init__(self):
        # the board is represented by a n * n array,
        # 1 represents chessman of the player who moves first,
        # -1 represents chessman of another player
        # 0 represents empty position
        self.data = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.winner = None
        self.hashVal = None
        self.end = None

    # calculate the hash value for one state, it's unique
    def getHash(self):
        if self.hashVal is None:
            self.hashVal = 0
            for i in self.data.reshape(BOARD_ROWS * BOARD_COLS):
                if i == -1:
                    i = 2
                self.hashVal = self.hashVal * 3 + i
        return int(self.hashVal)

    # determine whether a player has won the game, or it's a tie
    def isEnd(self):
        if self.end is not None:
            return self.end
        results = []
        # check row
        for i in range(0, BOARD_ROWS):
            results.append(np.sum(self.data[i, :]))
        # check columns
        for i in range(0, BOARD_COLS):
            results.append(np.sum(self.data[:, i]))

        # check diagonals
        results.append(0)
        for i in range(0, BOARD_ROWS):
            results[-1] += self.data[i, i]
        results.append(0)
        for i in range(0, BOARD_ROWS):
            results[-1] += self.data[i, BOARD_ROWS - 1 - i]

        for result in results:
            if result == 3:
                self.winner = 1
                self.end = True
                return self.end
            if result == -3:
                self.winner = -1
                self.end = True
                return self.end

        # whether it's a tie
        sum = np.sum(np.abs(self.data))
        if sum == BOARD_ROWS * BOARD_COLS:
            self.winner = 0
            self.end = True
            return self.end

        # game is still going on
        self.end = False
        return self.end

    # @symbol 1 or -1
    # put chessman symbol in position (i, j)
    def nextState(self, i, j, symbol):
        newState = State()
        newState.data = np.copy(self.data)
        newState.data[i, j] = symbol
        return newState

    # print the board
    def show(self):
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.data[i, j] == 1:
                    token = '*'
                if self.data[i, j] == 0:
                    token = '0'
                if self.data[i, j] == -1:
                    token = 'x'
                out += token + ' | '
            print(out)
        print('-------------')

In [4]:
def getAllStatesImpl(currentState, currentSymbol, allStates):
    for i in range(0, BOARD_ROWS):
        for j in range(0, BOARD_COLS):
            if currentState.data[i][j] == 0:
                newState = currentState.nextState(i, j, currentSymbol)
                newHash = newState.getHash()
                if newHash not in allStates.keys():
                    isEnd = newState.isEnd()
                    allStates[newHash] = (newState, isEnd)
                    if not isEnd:
                        getAllStatesImpl(newState, -currentSymbol, allStates)

In [5]:
def getAllStates():
    currentSymbol = 1
    currentState = State()
    allStates = dict()
    allStates[currentState.getHash()] = (currentState, currentState.isEnd())
    getAllStatesImpl(currentState, currentSymbol, allStates)
    return allStates

# all possible board configurations
allStates = getAllStates()

In [11]:
print (len(allStates))
allStates[5][0].show()

5478
-------------
| 0 | 0 | 0 | 
-------------
| 0 | 0 | 0 | 
-------------
| 0 | * | x | 
-------------


In [12]:
class Judger:
    # @player1: player who will move first, its chessman will be 1
    # @player2: another player with chessman -1
    # @feedback: if True, both players will receive rewards when game is end
    def __init__(self, player1, player2, feedback=True):
        self.p1 = player1
        self.p2 = player2
        self.feedback = feedback
        self.currentPlayer = None
        self.p1Symbol = 1
        self.p2Symbol = -1
        self.p1.setSymbol(self.p1Symbol)
        self.p2.setSymbol(self.p2Symbol)
        self.currentState = State()
        self.allStates = allStates

    # give reward to two players
    def giveReward(self):
        if self.currentState.winner == self.p1Symbol:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif self.currentState.winner == self.p2Symbol:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            self.p1.feedReward(0.1)
            self.p2.feedReward(0.5)

    def feedCurrentState(self):
        self.p1.feedState(self.currentState)
        self.p2.feedState(self.currentState)

    def reset(self):
        self.p1.reset()
        self.p2.reset()
        self.currentState = State()
        self.currentPlayer = None

    # @show: if True, print each board during the game
    def play(self, show=False):
        self.reset()
        self.feedCurrentState()
        while True:
            # set current player
            if self.currentPlayer == self.p1:
                self.currentPlayer = self.p2
            else:
                self.currentPlayer = self.p1
            if show:
                self.currentState.show()
            [i, j, symbol] = self.currentPlayer.takeAction()
            self.currentState = self.currentState.nextState(i, j, symbol)
            hashValue = self.currentState.getHash()
            self.currentState, isEnd = self.allStates[hashValue]
            self.feedCurrentState()
            if isEnd:
                if self.feedback:
                    self.giveReward()
                return self.currentState.winner

In [13]:
# AI player
class Player:
    # @stepSize: step size to update estimations
    # @exploreRate: possibility to explore
    def __init__(self, stepSize = 0.1, exploreRate=0.1):
        self.allStates = allStates
        self.estimations = dict()
        self.stepSize = stepSize
        self.exploreRate = exploreRate
        self.states = []

    def reset(self):
        self.states = []

    def setSymbol(self, symbol):
        self.symbol = symbol
        for hash in self.allStates.keys():
            (state, isEnd) = self.allStates[hash]
            if isEnd:
                if state.winner == self.symbol:
                    self.estimations[hash] = 1.0
                else:
                    self.estimations[hash] = 0
            else:
                self.estimations[hash] = 0.5

    # accept a state
    def feedState(self, state):
        self.states.append(state)

    # update estimation according to reward
    def feedReward(self, reward):
        if len(self.states) == 0:
            return
        self.states = [state.getHash() for state in self.states]
        target = reward
        for latestState in reversed(self.states):
            value = self.estimations[latestState] + self.stepSize * (target - self.estimations[latestState])
            self.estimations[latestState] = value
            target = value
        self.states = []

    # determine next action
    def takeAction(self):
        state = self.states[-1]
        nextStates = []
        nextPositions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if state.data[i, j] == 0:
                    nextPositions.append([i, j])
                    nextStates.append(state.nextState(i, j, self.symbol).getHash())
        if np.random.binomial(1, self.exploreRate):
            np.random.shuffle(nextPositions)
            # Not sure if truncating is the best way to deal with exploratory step
            # Maybe it's better to only skip this step rather than forget all the history
            self.states = []
            action = nextPositions[0]
            action.append(self.symbol)
            return action

        values = []
        for hash, pos in zip(nextStates, nextPositions):
            values.append((self.estimations[hash], pos))
        np.random.shuffle(values)
        values.sort(key=lambda x: x[0], reverse=True)
        action = values[0][1]
        action.append(self.symbol)
        return action

    def savePolicy(self):
        fw = open('optimal_policy_' + str(self.symbol), 'wb')
        pickle.dump(self.estimations, fw)
        fw.close()

    def loadPolicy(self):
        fr = open('optimal_policy_' + str(self.symbol),'rb')
        self.estimations = pickle.load(fr)
        fr.close()

In [14]:
# human interface
# input a number to put a chessman
# | 1 | 2 | 3 |
# | 4 | 5 | 6 |
# | 7 | 8 | 9 |
class HumanPlayer:
    def __init__(self, stepSize = 0.1, exploreRate=0.1):
        self.symbol = None
        self.currentState = None
        return
    def reset(self):
        return
    def setSymbol(self, symbol):
        self.symbol = symbol
        return
    def feedState(self, state):
        self.currentState = state
        return
    def feedReward(self, reward):
        return
    def takeAction(self):
        data = int(input("Input your position:"))
        data -= 1
        i = data // int(BOARD_COLS)
        j = data % BOARD_COLS
        if self.currentState.data[i, j] != 0:
            return self.takeAction()
        return (i, j, self.symbol)

In [15]:
def train(epochs=20000):
    player1 = Player()
    player2 = Player()
    judger = Judger(player1, player2)
    player1Win = 0.0
    player2Win = 0.0
    for i in range(0, epochs):
        print("Epoch", i)
        winner = judger.play()
        if winner == 1:
            player1Win += 1
        if winner == -1:
            player2Win += 1
        judger.reset()
    print(player1Win / epochs)
    print(player2Win / epochs)
    player1.savePolicy()
    player2.savePolicy()

In [17]:
def compete(turns=500):
    player1 = Player(exploreRate=0)
    player2 = Player(exploreRate=0)
    judger = Judger(player1, player2, False)
    player1.loadPolicy()
    player2.loadPolicy()
    player1Win = 0.0
    player2Win = 0.0
    for i in range(0, turns):
        print("Epoch", i)
        winner = judger.play()
        if winner == 1:
            player1Win += 1
        if winner == -1:
            player2Win += 1
        judger.reset()
    print(player1Win / turns)
    print(player2Win / turns)

In [18]:
def play():
    while True:
        player1 = Player(exploreRate=0)
        player2 = HumanPlayer()
        judger = Judger(player1, player2, False)
        player1.loadPolicy()
        winner = judger.play(True)
        if winner == player2.symbol:
            print("Win!")
        elif winner == player1.symbol:
            print("Lose!")
        else:
            print("Tie!")

In [19]:
train()

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110


Epoch 923
Epoch 924
Epoch 925
Epoch 926
Epoch 927
Epoch 928
Epoch 929
Epoch 930
Epoch 931
Epoch 932
Epoch 933
Epoch 934
Epoch 935
Epoch 936
Epoch 937
Epoch 938
Epoch 939
Epoch 940
Epoch 941
Epoch 942
Epoch 943
Epoch 944
Epoch 945
Epoch 946
Epoch 947
Epoch 948
Epoch 949
Epoch 950
Epoch 951
Epoch 952
Epoch 953
Epoch 954
Epoch 955
Epoch 956
Epoch 957
Epoch 958
Epoch 959
Epoch 960
Epoch 961
Epoch 962
Epoch 963
Epoch 964
Epoch 965
Epoch 966
Epoch 967
Epoch 968
Epoch 969
Epoch 970
Epoch 971
Epoch 972
Epoch 973
Epoch 974
Epoch 975
Epoch 976
Epoch 977
Epoch 978
Epoch 979
Epoch 980
Epoch 981
Epoch 982
Epoch 983
Epoch 984
Epoch 985
Epoch 986
Epoch 987
Epoch 988
Epoch 989
Epoch 990
Epoch 991
Epoch 992
Epoch 993
Epoch 994
Epoch 995
Epoch 996
Epoch 997
Epoch 998
Epoch 999
Epoch 1000
Epoch 1001
Epoch 1002
Epoch 1003
Epoch 1004
Epoch 1005
Epoch 1006
Epoch 1007
Epoch 1008
Epoch 1009
Epoch 1010
Epoch 1011
Epoch 1012
Epoch 1013
Epoch 1014
Epoch 1015
Epoch 1016
Epoch 1017
Epoch 1018
Epoch 1019
Epoch 1020

Epoch 1832
Epoch 1833
Epoch 1834
Epoch 1835
Epoch 1836
Epoch 1837
Epoch 1838
Epoch 1839
Epoch 1840
Epoch 1841
Epoch 1842
Epoch 1843
Epoch 1844
Epoch 1845
Epoch 1846
Epoch 1847
Epoch 1848
Epoch 1849
Epoch 1850
Epoch 1851
Epoch 1852
Epoch 1853
Epoch 1854
Epoch 1855
Epoch 1856
Epoch 1857
Epoch 1858
Epoch 1859
Epoch 1860
Epoch 1861
Epoch 1862
Epoch 1863
Epoch 1864
Epoch 1865
Epoch 1866
Epoch 1867
Epoch 1868
Epoch 1869
Epoch 1870
Epoch 1871
Epoch 1872
Epoch 1873
Epoch 1874
Epoch 1875
Epoch 1876
Epoch 1877
Epoch 1878
Epoch 1879
Epoch 1880
Epoch 1881
Epoch 1882
Epoch 1883
Epoch 1884
Epoch 1885
Epoch 1886
Epoch 1887
Epoch 1888
Epoch 1889
Epoch 1890
Epoch 1891
Epoch 1892
Epoch 1893
Epoch 1894
Epoch 1895
Epoch 1896
Epoch 1897
Epoch 1898
Epoch 1899
Epoch 1900
Epoch 1901
Epoch 1902
Epoch 1903
Epoch 1904
Epoch 1905
Epoch 1906
Epoch 1907
Epoch 1908
Epoch 1909
Epoch 1910
Epoch 1911
Epoch 1912
Epoch 1913
Epoch 1914
Epoch 1915
Epoch 1916
Epoch 1917
Epoch 1918
Epoch 1919
Epoch 1920
Epoch 1921
Epoch 1922

Epoch 2738
Epoch 2739
Epoch 2740
Epoch 2741
Epoch 2742
Epoch 2743
Epoch 2744
Epoch 2745
Epoch 2746
Epoch 2747
Epoch 2748
Epoch 2749
Epoch 2750
Epoch 2751
Epoch 2752
Epoch 2753
Epoch 2754
Epoch 2755
Epoch 2756
Epoch 2757
Epoch 2758
Epoch 2759
Epoch 2760
Epoch 2761
Epoch 2762
Epoch 2763
Epoch 2764
Epoch 2765
Epoch 2766
Epoch 2767
Epoch 2768
Epoch 2769
Epoch 2770
Epoch 2771
Epoch 2772
Epoch 2773
Epoch 2774
Epoch 2775
Epoch 2776
Epoch 2777
Epoch 2778
Epoch 2779
Epoch 2780
Epoch 2781
Epoch 2782
Epoch 2783
Epoch 2784
Epoch 2785
Epoch 2786
Epoch 2787
Epoch 2788
Epoch 2789
Epoch 2790
Epoch 2791
Epoch 2792
Epoch 2793
Epoch 2794
Epoch 2795
Epoch 2796
Epoch 2797
Epoch 2798
Epoch 2799
Epoch 2800
Epoch 2801
Epoch 2802
Epoch 2803
Epoch 2804
Epoch 2805
Epoch 2806
Epoch 2807
Epoch 2808
Epoch 2809
Epoch 2810
Epoch 2811
Epoch 2812
Epoch 2813
Epoch 2814
Epoch 2815
Epoch 2816
Epoch 2817
Epoch 2818
Epoch 2819
Epoch 2820
Epoch 2821
Epoch 2822
Epoch 2823
Epoch 2824
Epoch 2825
Epoch 2826
Epoch 2827
Epoch 2828

Epoch 3609
Epoch 3610
Epoch 3611
Epoch 3612
Epoch 3613
Epoch 3614
Epoch 3615
Epoch 3616
Epoch 3617
Epoch 3618
Epoch 3619
Epoch 3620
Epoch 3621
Epoch 3622
Epoch 3623
Epoch 3624
Epoch 3625
Epoch 3626
Epoch 3627
Epoch 3628
Epoch 3629
Epoch 3630
Epoch 3631
Epoch 3632
Epoch 3633
Epoch 3634
Epoch 3635
Epoch 3636
Epoch 3637
Epoch 3638
Epoch 3639
Epoch 3640
Epoch 3641
Epoch 3642
Epoch 3643
Epoch 3644
Epoch 3645
Epoch 3646
Epoch 3647
Epoch 3648
Epoch 3649
Epoch 3650
Epoch 3651
Epoch 3652
Epoch 3653
Epoch 3654
Epoch 3655
Epoch 3656
Epoch 3657
Epoch 3658
Epoch 3659
Epoch 3660
Epoch 3661
Epoch 3662
Epoch 3663
Epoch 3664
Epoch 3665
Epoch 3666
Epoch 3667
Epoch 3668
Epoch 3669
Epoch 3670
Epoch 3671
Epoch 3672
Epoch 3673
Epoch 3674
Epoch 3675
Epoch 3676
Epoch 3677
Epoch 3678
Epoch 3679
Epoch 3680
Epoch 3681
Epoch 3682
Epoch 3683
Epoch 3684
Epoch 3685
Epoch 3686
Epoch 3687
Epoch 3688
Epoch 3689
Epoch 3690
Epoch 3691
Epoch 3692
Epoch 3693
Epoch 3694
Epoch 3695
Epoch 3696
Epoch 3697
Epoch 3698
Epoch 3699

...

Epoch 16606
Epoch 16607
Epoch 16608
Epoch 16609
Epoch 16610
Epoch 16611
Epoch 16612
Epoch 16613
Epoch 16614
Epoch 16615
Epoch 16616
Epoch 16617
Epoch 16618
Epoch 16619
Epoch 16620
Epoch 16621
Epoch 16622
Epoch 16623
Epoch 16624
Epoch 16625
Epoch 16626
Epoch 16627
Epoch 16628
Epoch 16629
Epoch 16630
Epoch 16631
Epoch 16632
Epoch 16633
Epoch 16634
Epoch 16635
Epoch 16636
Epoch 16637
Epoch 16638
Epoch 16639
Epoch 16640
Epoch 16641
Epoch 16642
Epoch 16643
Epoch 16644
Epoch 16645
Epoch 16646
Epoch 16647
Epoch 16648
Epoch 16649
Epoch 16650
Epoch 16651
Epoch 16652
Epoch 16653
Epoch 16654
Epoch 16655
Epoch 16656
Epoch 16657
Epoch 16658
Epoch 16659
Epoch 16660
Epoch 16661
Epoch 16662
Epoch 16663
Epoch 16664
Epoch 16665
Epoch 16666
Epoch 16667
Epoch 16668
Epoch 16669
Epoch 16670
Epoch 16671
Epoch 16672
Epoch 16673
Epoch 16674
Epoch 16675
Epoch 16676
Epoch 16677
Epoch 16678
Epoch 16679
Epoch 16680
Epoch 16681
Epoch 16682
Epoch 16683
Epoch 16684
Epoch 16685
Epoch 16686
Epoch 16687
Epoch 16688
Epoc

Epoch 17443
Epoch 17444
Epoch 17445
Epoch 17446
Epoch 17447
Epoch 17448
Epoch 17449
Epoch 17450
Epoch 17451
Epoch 17452
Epoch 17453
Epoch 17454
Epoch 17455
Epoch 17456
Epoch 17457
Epoch 17458
Epoch 17459
Epoch 17460
Epoch 17461
Epoch 17462
Epoch 17463
Epoch 17464
Epoch 17465
Epoch 17466
Epoch 17467
Epoch 17468
Epoch 17469
Epoch 17470
Epoch 17471
Epoch 17472
Epoch 17473
Epoch 17474
Epoch 17475
Epoch 17476
Epoch 17477
Epoch 17478
Epoch 17479
Epoch 17480
Epoch 17481
Epoch 17482
Epoch 17483
Epoch 17484
Epoch 17485
Epoch 17486
Epoch 17487
Epoch 17488
Epoch 17489
Epoch 17490
Epoch 17491
Epoch 17492
Epoch 17493
Epoch 17494
Epoch 17495
Epoch 17496
Epoch 17497
Epoch 17498
Epoch 17499
Epoch 17500
Epoch 17501
Epoch 17502
Epoch 17503
Epoch 17504
Epoch 17505
Epoch 17506
Epoch 17507
Epoch 17508
Epoch 17509
Epoch 17510
Epoch 17511
Epoch 17512
Epoch 17513
Epoch 17514
Epoch 17515
Epoch 17516
Epoch 17517
Epoch 17518
Epoch 17519
Epoch 17520
Epoch 17521
Epoch 17522
Epoch 17523
Epoch 17524
Epoch 17525
Epoc

Epoch 18296
Epoch 18297
Epoch 18298
Epoch 18299
Epoch 18300
Epoch 18301
Epoch 18302
Epoch 18303
Epoch 18304
Epoch 18305
Epoch 18306
Epoch 18307
Epoch 18308
Epoch 18309
Epoch 18310
Epoch 18311
Epoch 18312
Epoch 18313
Epoch 18314
Epoch 18315
Epoch 18316
Epoch 18317
Epoch 18318
Epoch 18319
Epoch 18320
Epoch 18321
Epoch 18322
Epoch 18323
Epoch 18324
Epoch 18325
Epoch 18326
Epoch 18327
Epoch 18328
Epoch 18329
Epoch 18330
Epoch 18331
Epoch 18332
Epoch 18333
Epoch 18334
Epoch 18335
Epoch 18336
Epoch 18337
Epoch 18338
Epoch 18339
Epoch 18340
Epoch 18341
Epoch 18342
Epoch 18343
Epoch 18344
Epoch 18345
Epoch 18346
Epoch 18347
Epoch 18348
Epoch 18349
Epoch 18350
Epoch 18351
Epoch 18352
Epoch 18353
Epoch 18354
Epoch 18355
Epoch 18356
Epoch 18357
Epoch 18358
Epoch 18359
Epoch 18360
Epoch 18361
Epoch 18362
Epoch 18363
Epoch 18364
Epoch 18365
Epoch 18366
Epoch 18367
Epoch 18368
Epoch 18369
Epoch 18370
Epoch 18371
Epoch 18372
Epoch 18373
Epoch 18374
Epoch 18375
Epoch 18376
Epoch 18377
Epoch 18378
Epoc

Epoch 19143
Epoch 19144
Epoch 19145
Epoch 19146
Epoch 19147
Epoch 19148
Epoch 19149
Epoch 19150
Epoch 19151
Epoch 19152
Epoch 19153
Epoch 19154
Epoch 19155
Epoch 19156
Epoch 19157
Epoch 19158
Epoch 19159
Epoch 19160
Epoch 19161
Epoch 19162
Epoch 19163
Epoch 19164
Epoch 19165
Epoch 19166
Epoch 19167
Epoch 19168
Epoch 19169
Epoch 19170
Epoch 19171
Epoch 19172
Epoch 19173
Epoch 19174
Epoch 19175
Epoch 19176
Epoch 19177
Epoch 19178
Epoch 19179
Epoch 19180
Epoch 19181
Epoch 19182
Epoch 19183
Epoch 19184
Epoch 19185
Epoch 19186
Epoch 19187
Epoch 19188
Epoch 19189
Epoch 19190
Epoch 19191
Epoch 19192
Epoch 19193
Epoch 19194
Epoch 19195
Epoch 19196
Epoch 19197
Epoch 19198
Epoch 19199
Epoch 19200
Epoch 19201
Epoch 19202
Epoch 19203
Epoch 19204
Epoch 19205
Epoch 19206
Epoch 19207
Epoch 19208
Epoch 19209
Epoch 19210
Epoch 19211
Epoch 19212
Epoch 19213
Epoch 19214
Epoch 19215
Epoch 19216
Epoch 19217
Epoch 19218
Epoch 19219
Epoch 19220
Epoch 19221
Epoch 19222
Epoch 19223
Epoch 19224
Epoch 19225
Epoc

Epoch 19973
Epoch 19974
Epoch 19975
Epoch 19976
Epoch 19977
Epoch 19978
Epoch 19979
Epoch 19980
Epoch 19981
Epoch 19982
Epoch 19983
Epoch 19984
Epoch 19985
Epoch 19986
Epoch 19987
Epoch 19988
Epoch 19989
Epoch 19990
Epoch 19991
Epoch 19992
Epoch 19993
Epoch 19994
Epoch 19995
Epoch 19996
Epoch 19997
Epoch 19998
Epoch 19999
0.21765
0.24605


In [20]:
compete()

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110


In [21]:
play()

-------------
| 0 | 0 | 0 | 
-------------
| 0 | 0 | 0 | 
-------------
| 0 | 0 | 0 | 
-------------
-------------
| 0 | 0 | 0 | 
-------------
| 0 | * | 0 | 
-------------
| 0 | 0 | 0 | 
-------------
Input your position:1
-------------
| x | 0 | 0 | 
-------------
| 0 | * | 0 | 
-------------
| 0 | 0 | 0 | 
-------------
-------------
| x | 0 | 0 | 
-------------
| 0 | * | 0 | 
-------------
| 0 | 0 | * | 
-------------
Input your position:3
-------------
| x | 0 | x | 
-------------
| 0 | * | 0 | 
-------------
| 0 | 0 | * | 
-------------
-------------
| x | * | x | 
-------------
| 0 | * | 0 | 
-------------
| 0 | 0 | * | 
-------------
Input your position:6
-------------
| x | * | x | 
-------------
| 0 | * | x | 
-------------
| 0 | 0 | * | 
-------------
Lose!
-------------
| 0 | 0 | 0 | 
-------------
| 0 | 0 | 0 | 
-------------
| 0 | 0 | 0 | 
-------------
-------------
| 0 | 0 | 0 | 
-------------
| 0 | * | 0 | 
-------------
| 0 | 0 | 0 | 
-------------


KeyboardInterrupt: 